In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.impute import SimpleImputer
from xgboost import XGBRegressor

from fastapi import FastAPI, UploadFile, File
from io import BytesIO

import pandas as pd
import joblib

In [ ]:
df = pd.read_csv('Laptop_price.csv')
X = df.drop(columns=['Price'])
y = df['Price']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
num_features = X.select_dtypes(include=['int64', 'float64']).columns.tolist()
cat_features = X.select_dtypes(include=['object']).columns.tolist()
num_transformer = Pipeline([
    ('imputer', SimpleImputer(strategy='median')),
    ('scaler', StandardScaler())
])
cat_transformer = Pipeline([    
        ('imputer', SimpleImputer(strategy='most_frequent')), 
        ('encoder', OneHotEncoder(handle_unknown='ignore'))
])
preprocessor = ColumnTransformer([
    ('num', num_transformer, num_features),
    ('cat', cat_transformer, cat_features)
])
pipeline = Pipeline([
    ('preprocessor', preprocessor), 
   ('model', XGBRegressor(n_estimators=100, learning_rate=0.1, max_depth=5))
])
pipeline.fit(X_train, y_train)
joblib.dump(pipeline, 'laptop_price_model.pkl')

['laptop_price_model.pkl']

In [ ]:
!git init 

Reinitialized existing Git repository in /Users/matthewandrews/Documents/Study MTUCI/Informatics Technologies/.git/


In [ ]:
!git add pipline.ipynb
!git add Laptop_price.csv

In [ ]:
!git commit -m "Добавлен ML-пайплайн"

[main 44b2c45] Добавлен ML-пайплайн
 1 file changed, 7 insertions(+), 28 deletions(-)


In [ ]:
!git remote add origin https://github.com/Liroyz/lab_pipline-

error: remote origin already exists.


In [ ]:
!git push -u origin main

Enumerating objects: 5, done.
Counting objects: 100% (5/5), done.
Delta compression using up to 8 threads
Compressing objects: 100% (3/3), done.
Writing objects: 100% (3/3), 436 bytes | 436.00 KiB/s, done.
Total 3 (delta 1), reused 0 (delta 0), pack-reused 0
remote: Resolving deltas: 100% (1/1), completed with 1 local object.
To https://github.com/Liroyz/lab_pipline-
   88c1e62..44b2c45  main -> main
branch 'main' set up to track 'origin/main'.


In [ ]:
%%writefile app.py

app = FastAPI()

# Загрузка обученной модели
model_path = "laptop_price_model.pkl"
model = joblib.load(model_path)

@app.post("/predict/")
async def predict(file: UploadFile = File(...)):
    content = await file.read()
    df = pd.read_csv(BytesIO(content))
    predictions = model.predict(df)
    return {"predictions": predictions.tolist()}

Overwriting app.py


In [ ]:
# !nohup uvicorn app:app --host 0.0.0.0 --port 8000 --reload > fastapi.log 2>&1 &

